### Todo

#### This cell is for defining various OPTIONS used for this notebook (working directory, how many rows and columns pandas displays for a dataframe, etc). 

#### Preferably this cell is also where we do important imports (for example pandas and numpy)

In [28]:
import os 
#Input the directory where your joined_data.csv is located 
#os.chdir('C:/Users/Trond/Documents/Master 2020/Processed data')
#os.chdir('C:/Users/Briggstone/Documents/Master 2020/Processed data')
os.chdir('C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data')

#Where you want the csv file of the merged data to be placed
#output_filepath = 'C:/Users/Briggstone/Documents/Master 2020/Processed data'
output_filepath = 'C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data'

import pandas as pd 
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)


#### In this cell we import our training data, convert HALL into HALL_EVER and select only BL observations.

In [53]:
train = pd.read_csv('train.csv')
temptrain = pd.DataFrame(train.PATNO.unique(), columns = ["PATNO"])

HALL_EVER = []
for id in train.PATNO.unique():
    if train.loc[(train.PATNO == id) & (train.HALL == 1), "HALL"].empty:
        HALL_EVER.append(0)
    else:
        HALL_EVER.append(1)

temptrain["HALL_EVER"] = HALL_EVER
train.drop("HALL", axis = 1, inplace = True)
train = train.merge(temptrain, how = "inner", on = "PATNO")

#Selecting only Baseline observations
train = train.loc[train.EVENT_ID == "BL", :]

#We can then safely drop EVENT_ID
train.drop("EVENT_ID", axis = 1, inplace = True)

# We form Y
Y = train.pop("HALL_EVER")

In [54]:
train.shape

(293, 21)

#### In this cell we apply random forest from XGBOOST

In [ ]:
dtrain = xgb.DMatrix(train, label = Y)

params = {
  'colsample_bynode': 0.8,
  'learning_rate': 1,
  'max_depth': 5,
  'num_parallel_tree': 100,
  'objective': 'binary:logistic',
  'subsample': 0.8,
  'tree_method': 'gpu_hist'
}

cv_results = xgb.cv(params, dtrain = dtrain, num_boost_round=1, nfold = 10, as_pandas = True, seed = 1, metrics = ["error", "auc"])

In [ ]:
cv_results

#### In this cell we apply SVM from scikit-learn

In [62]:
clf1 = svm.SVC(random_state=0, gamma='auto', kernel='rbf')
roc_auc1 = cross_val_score(clf1, train, Y, cv=10, scoring='roc_auc')
roc_auc1

array([0.57954545, 0.49431818, 0.43181818, 0.42857143, 0.51190476,
       0.45833333, 0.57142857, 0.43452381, 0.31547619, 0.28571429])

In [63]:
accuracy1 = cross_val_score(clf1, train, Y, cv=10, scoring='accuracy')
accuracy1

array([0.73333333, 0.73333333, 0.63333333, 0.68965517, 0.72413793,
       0.68965517, 0.72413793, 0.72413793, 0.72413793, 0.72413793])

In [64]:
clf2 = svm.SVC(random_state=0, gamma='auto', kernel='linear')
roc_auc2 = cross_val_score(clf2, train, Y, cv=10, scoring='roc_auc')
roc_auc2

array([0.69318182, 0.67613636, 0.65909091, 0.44047619, 0.6547619 ,
       0.64285714, 0.70833333, 0.55357143, 0.79166667, 0.66071429])

In [65]:
accuracy2 = cross_val_score(clf2, train, Y, cv=10, scoring='accuracy')
accuracy2

array([0.66666667, 0.76666667, 0.7       , 0.62068966, 0.75862069,
       0.65517241, 0.68965517, 0.72413793, 0.75862069, 0.75862069])